In [2]:
import tensorflow as tf
import numpy as np

There are two important objects in tensorflow: variable and tensor.

Variable is a variable as we know it. It's a wrapping class of a np array and always store some definite values (it's more versatile than a np array). It's intuitive. It has fixed, unchangeable shape and type (and is rather strict about types), and a variable value. However, it's possible to initialize shape = tf.TensorShape(None). Then shape will be inferred from the first assignment.  

Tensor is all tensorflow is based on. Originally, everything in tensorflow is a graph, from model building to training and evaluating. In other words, all executions are lazy. When we do a+b, tf doesn't just add the two values, it records the addition as an operation along the graph. When we do run, it executes the graph and compute actual values. This is the case when eager execution is not on (in tf 2.0 it's by default on)

A Tensor is such a latent variable. It may store definite numeric value, or store its dependencies on previous tensors (how to compute the value of this tensor given the value of previous tensors. or it may trace the dependency up to the very first variable (express this tensor value in terms of the value of the original input tensor values)). So in other words, a tensor stores an expression for computing its value.

So, that's why a layer takes in a tensor and outputs a tensor. 

A model also takes in a tensor and outputs a tensor.

model.input and model.output are both tensors

Tensors are the core of a model or layer (with a few other parameters), so it's not surprising that we can just build new models out of old ones by passing a tensor into the model. 

In [3]:
stringVar = tf.Variable('helloworld',tf.string)
intVar = tf.Variable([12,3,4],tf.int32)
floatVar = tf.Variable([[2,23,2],[3,2,1.0],[3,2,54]],tf.float32)
complexVar = tf.Variable([1+2j,2-3j],tf.complex128) # here j represents mathematical i
varFromTensor = tf.Variable(tf.constant(3.0,shape=(3,3)))
# Obviously your tensor should be evaluatable eagerly, else you get Nones?
print(stringVar)
print(intVar)
print(floatVar)
print(complexVar)
print(varFromTensor)
print(floatVar+varFromTensor)
# All operations +,-,*,/ etc returns a tensor in tensorflow
# unless we use special variable operations
intVar.assign([1,2,3])
# when we use assign, shape should be the same
floatVar.assign_add(floatVar)
intVar.assign_sub(3*tf.ones(3,dtype=tf.int32))

<tf.Variable 'Variable:0' shape=() dtype=string, numpy=b'helloworld'>
<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([12,  3,  4], dtype=int32)>
<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[ 2., 23.,  2.],
       [ 3.,  2.,  1.],
       [ 3.,  2., 54.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=complex128, numpy=array([1.+2.j, 2.-3.j])>
<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[3., 3., 3.],
       [3., 3., 3.],
       [3., 3., 3.]], dtype=float32)>
tf.Tensor(
[[ 5. 26.  5.]
 [ 6.  5.  4.]
 [ 6.  5. 57.]], shape=(3, 3), dtype=float32)


<tf.Variable 'UnreadVariable' shape=(3,) dtype=int32, numpy=array([-2, -1,  0], dtype=int32)>

In [4]:
tensor1 = tf.constant([1,2,3,4],dtype=tf.float64)
# for tensors with definite values, we can use tensor.numpy() to get its np values
print(tensor1.numpy())
tensor2 = tf.constant([1,2,3,4],dtype=tf.float64,shape=(1,2,2,1))
# shape basically reshapes the input list or np array
print(tensor2.numpy())

[1. 2. 3. 4.]
[[[[1.]
   [2.]]

  [[3.]
   [4.]]]]


In [5]:
print(tf.rank(tensor2))
# rank gives the space this tensor is in (or its # of dimensions)
tensor3 = tf.reshape(tensor2, shape=(4,1))
# tensor operators has mostly everything numpy has
zeros = tf.zeros((2,3,4))
ones = tf.ones((3,3,4))
eye = tf.eye(4)*8
print(eye.numpy())

tf.Tensor(4, shape=(), dtype=int32)
[[8. 0. 0. 0.]
 [0. 8. 0. 0.]
 [0. 0. 8. 0.]
 [0. 0. 0. 8.]]


In [6]:
concate = tf.concat([zeros,ones],0)
print(concate.numpy())

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


In [11]:
tf.config.list_physical_devices()
# list all the cpus and gpus accessible
# tensorflow automatically allocates a tensor operation to cpu or gpu

/job:localhost/replica:0/task:0/device:CPU:0


In [12]:
concate.device
# check device used by a tensor through tensor.device (note we are allocating tensors to devices 
# because tensors are expressions that ultimately need to be computed, 
# so an even allocation of tensors to devices can ensure the fast speed when we finally execute the graph)

# CPU:#, # means the index of the cpu or gpu used

# GPUs are fast for matrix addition while CPUs are fast for matrix multiplication

/job:localhost/replica:0/task:0/device:CPU:0


In [14]:
# we can enforce the usage of a particular core by 
# tf.device('CPU:0') function
with tf.device('CPU:0'):
    a = tf.ones((3,3))
    # using with so that this enforcement ends after the block